# Xray analyzer
In this notebook we read in the data and visualize it to see it is clean,etc.

At the end the data gets written away to train the model.

In early versions this will all be done in the same notebook.

## Import and checks

In [ ]:
# Import env libraries
import torch
from torchvision import datasets, transforms
import os
import sys

# Local libraries
sys.path.append(os.path.dirname(os.getcwd()))
from src import model_def as md
from src import data_processing as dp
from src import model_eval as me

In [ ]:
# check if Pytorch CUDA is installed correctly

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))

print("CUDA Version:", torch.version.cuda)
print("cuDNN Version:", torch.backends.cudnn.version())

## Sorting

### Pre-sorting
Take a test/train sampleset and put it into the respective folder, according to wanted Ns and the split between test and train.

In [ ]:
dp.sort_images_train_test(N=13000, delete_test_train_folder=True)

### Label Sorting

Sort data accoridng to provided label of the given classification of the dataset (data_entry_2017.csv). This is needed for training single outcome models

In [ ]:
csv_path = os.path.join("..", "data", "classification", "Data_Entry_2017_v2020.csv")
image_folder = os.path.join("..", "data", "images_train")
output_folder = os.path.join("..", "data", "images_sorted_train")

# dp.organize_images_by_label_folder(csv_path, image_folder, output_folder)

image_folder = os.path.join("..", "data", "images_test")
output_folder = os.path.join("..", "data", "images_sorted_test")

# dp.organize_images_by_label_folder(csv_path, image_folder, output_folder)

#### Data loading

Import a testset to be used in training

We normalize the input since we work in grayscale. No size transformations are done since the dataset is already in 1024X1024.

This is only needed for the single outcome models.

TODO: figure out if we need normalizing in the multi-model

In [ ]:

train_image_folder = os.path.join("..", "data", "images_sorted_train")
test_image_folder = os.path.join("..", "data", "images_sorted_test")
# print(train_image_folder)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.Grayscale(num_output_channels=1), 
])


# dataset_xrays_train = datasets.ImageFolder(root=train_image_folder, transform=transform)
# dataset_xrays_test = datasets.ImageFolder(root=test_image_folder, transform=transform)

# print(dataset_xrays_train)

#### Model setup
Below the actual model is trained

In [ ]:
model, data_loader, criterion, optimizer, device = md.setup_model_and_training(None, model_type="MultiAttention", batch_size=16, model_mode="multi", learning_rate=0.001)
# data_loader_test = md.XrayMultiLabelDataset(os.path.join("data", "images_test"))
# model = md.train_model(model, data_loader, criterion, optimizer, device, num_epochs=5)
model = md.train_multilabel_model(model, data_loader, criterion, optimizer, device, num_epochs=100)


#### Evaluate the model
First eval against the train set, then evala gainst test set

In [ ]:

metrics = me.evaluate_multi_model(model, data_loader, device, class_names=data_loader.dataset.get_labels())
# me.visualize_predictions(model, data_loader, device, num_examples=5, class_names=dataset_xrays_test.classes)
# misclass_df = me.analyze_misclassifications(model, data_loader, device, class_names=dataset_xrays_train.classes)
# hard_examples = me.find_hard_examples(model, data_loader, device, n=5)

Eval against test set

In [ ]:

data_loader_test = md.setup_dataloader(None, path_multi=os.path.join("..", "data", "images_test"), model_type="MultiAttention") # dataset_xrays_test
metrics = me.evaluate_multi_model(model, data_loader_test, device, class_names=data_loader_test.dataset.get_labels())
# me.visualize_predictions(model, data_loader_test, device, num_examples=5, class_names=dataset_xrays_test.classes)
# misclass_df = me.analyze_misclassifications(model, data_loader_test, device, class_names=dataset_xrays_test.classes)
# hard_examples = me.find_hard_examples(model, data_loader_test, device, n=5)